# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use pandas and numpy functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Import up Required Module create

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
PATH = 'data'
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [6]:
sales = pd.read_csv(f'{PATH}/kc_house_data.csv', dtype=dtype_dict)

# Split data into training and testing

In [32]:
train_data = pd.read_csv(f'{PATH}/kc_house_data.csv', dtype=dtype_dict)
test_data = pd.read_csv(f'{PATH}/kc_house_data.csv', dtype=dtype_dict)

# Useful summary functions

In order to make use of the closed form solution as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an Array
* Computing the arithmetic average (mean) of an Array
* multiplying Arrays by constants
* multiplying Arrays by other Arrays

In [10]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales.price

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print (f"average price via method 1: {avg_price_1}")
print (f"average price via method 2: {avg_price_2}")

average price via method 1: 540088.1417665294
average price via method 2: 540088.1417665294


In [12]:
half_prices = 0.5*prices
prices_squared = prices ** 2
sum_prices_squared = prices_squared.sum()
print(f'The sum of prices squared is: {sum_prices_squared}')

The sum of prices squared is: 9217325138472070.0


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [37]:
def simple_linear_regression(input_feature, output):
    n = len(output)
    
    numerator = (input_feature * output).sum() - ((input_feature.sum() * output.sum()) / n)
    denominator = ((input_feature**2).sum()) - ((input_feature.sum() * input_feature.sum()) / n)
    # use the formula for the slope
    slope = numerator / denominator
    # use the formula for the intercept
    intercept = output.mean() - (slope * input_feature.mean())
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [38]:
test_feature = np.array(range(5))
test_output = np.array(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print(f"Intercept: {test_intercept}")
print(f"Slope: {test_slope}")

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [39]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print (f"Intercept: {sqft_intercept}")
print (f"Slope: {sqft_slope}")

Intercept: -43580.743094473844
Slope: 280.6235678974482


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [40]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = input_feature * slope + intercept
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [42]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print(f"The estimated price for a house with {my_house_sqft} squarefeet is {estimated_price}")

The estimated price for a house with 2650 squarefeet is 700071.7118337639


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [43]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predictions = get_regression_predictions(input_feature, intercept, slope)
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - predictions
    # square the residuals and add them up
    RSS = residuals.T.dot(residuals)
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [45]:
print(get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)) # should be 0.0

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [46]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print(f'The RSS of predicting Prices based on Square Feet is : {rss_prices_on_sqft}') 

The RSS of predicting Prices based on Square Feet is : 1477276362322489.8


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [47]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = (output - intercept) / slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [48]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print(f"The estimated squarefeet for a house worth ${my_house_price} is {estimated_squarefeet}")

The estimated squarefeet for a house worth $800000 is 3006.093712708956


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [49]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'

sqft_intercept, sqft_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print (f"Intercept: {sqft_intercept}")
print (f"Slope: {sqft_slope}")

Intercept: 129802.35631825897
Slope: 121716.12651183833


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [53]:
# Compute RSS when using bedrooms on TEST data:
# Compute RSS when using squarefeet on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(test_data['sqft_living'], test_data['price'])
rss_prices_on_sqft = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], sqft_intercept, sqft_slope)
print (f'The RSS of predicting Prices based on Bedrooms is : {rss_prices_on_sqft}')

The RSS of predicting Prices based on Bedrooms is : 1.0250681822876348e+16


In [52]:
# Compute RSS when using squarefeet on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(test_data['bedrooms'], test_data['price'])
rss_prices_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], sqft_intercept, sqft_slope)
print (f'The RSS of predicting Prices based on Bedrooms is : {rss_prices_on_bedrooms}')

The RSS of predicting Prices based on Bedrooms is : 2635958166421228.0
